# Breast Cancer Classification

## Objective

We are predicting the likelihood for a patient getting diagnosed with a malignant form of breast cancer using the attributes of the dataset.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.style.use('ggplot')

import seaborn as sns

import xgboost as xgb
from sklearn.metrics import confusion_matrix, plot_roc_curve

sns.set(context='notebook',
        rc={'figure.figsize':(12,10)},
        palette=sns.color_palette('tab10', 10))

## 1. Getting the data
You can get this data set from this [link](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) and is loaded in this project as a `Data asset` for convenience.

In [ ]:
def get_data():
    data = pd.read_csv("/project_data/data_asset/data.csv")
    cols = list(data.columns[1:12])
    return data[cols]

In [ ]:
df = get_data()
cols = df.columns

This dataset contains 569 samples of malignant and benign tumor samples. The `diagnosis` column has two values, `M` represents malignant and `B` benign tumors. The rest of the columns, 2 to 11, contain 10 real valued features that have been computed from digitized of image cell nuclei which is used to build a model and predict whether the tumor is malignant or benign.

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.diagnosis.value_counts()

## 2. Data Visualization
We are using heatmaps to look at correlations between the attributes, the darker the cell color the higher the correlation between two variables. It is possible to notice that most attributes are highly correlated amongst themselves, for example `area_mean` and `perimeter_mean` are almost collinear.

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df[cols].corr(), annot=True, square=True, cmap='coolwarm')
plt.show()

Below is the pairwise plot of the attributes between each other. The plots on the diagonal show the density distribution of the attributes and the off-diagonal plots caputre the relationship between a pair of different attributes. Densitiy plots on the diagonal that are highly overlaping indicate that the variables have distribution very close to each other. Plots that have trend lines around the 45 degrees indicate that they have a strong correlation.

In [ ]:
sns.pairplot(df, hue = 'diagnosis')

## 3. Model training

We are going to use columns 2 to 11 as indipendent variables and we set the `diagnosis` as the target variable.

In [ ]:
X = df[cols[1:]]
y = df.diagnosis

We randomly divide the modeling dataset into a training (80%) and test (20%) and because of the inbalance in the class we are using a stratified random sample to separate the dataset into training and test groups.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

All the predictor data types are numerical values.

In [ ]:
X_train.dtypes

We are using a Extreme Boosted model (XGBoost). Since the dependent varialbe is a class variable, we are encoding it into a binary variable using a label encoder. During the training the model is evaluated using the `error` metric. Also note that we estimating the parameters only on the training dataset.

In [ ]:

xgb_clf = xgb.XGBClassifier(
            random_state=34,
            use_label_encoder=True,
            eval_metric="error"
)
xgb_clf.fit(X_train, y_train)

Now we are applying the trained model to predict on the test dataset `X_test`.

In [ ]:
predictions = xgb_clf.predict(X_test)

## 4. Model performance

We are using the confusion matrix to asses how the model performs on the test dataset. Since the model is predicting the likelihood of the presence of malignant tumor a threshold of 0.5 is automatically applied to the model predictions to turn them into 1 or 0. Columns represent the model predictions and rows the actual values in the dataset. The performance in the model is very good since it has captured 39 of the 42 malignant breast cancers with only 3 false positives.

In [ ]:
cm = confusion_matrix(y_test, predictions)
sns.heatmap(cm, annot=True,  annot_kws={"size": 24})


The ROC curve tells us that there's a 97% chance that the model will predict breast cancer for a patient that actually has breast cancer compared to a patient who doesn't have it.

In [ ]:
plot_roc_curve(xgb_clf, X_test, y_test)  
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r')